In [ ]:
! pip list | grep chromadb || pip install chromadb==0.5.2

In [ ]:
import gzip
import json
import os
import pprint
import re
import subprocess
import time
from datetime import timedelta, datetime
from pathlib import Path
from dataclasses import dataclass, asdict
import multiprocessing

import numpy as np
import pandas as pd
import requests

import chromadb

In [ ]:
@dataclass
class DataSetConfig:
    content_path: Path
    embedding_path: Path
    num_of_docs: int
    index_size: int
    bulk_size: int
    index_name: str
    distance: str
    dimension: int
    exact: bool
    hnsw_m: int
    hnsw_ef_construction: int
    hnsw_ef: int
    update_docs_per_sec: int
    quantization: str

    chroma_name: str = "benchmark_chroma"
    chroma_host: str = "localhost"
    chroma_port: int = 8008
    chroma_version: str = "0.5.2"


def get_dataset_config(target_name):
    setting = {
        "100k-768-m32-efc200-ef100-ip": {
            "content_path": Path("dataset/passages-c400-jawiki-20230403"),
            "embedding_path": Path("dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage"),
            "num_of_docs": 5555583,
            "index_size": 100000,
            "bulk_size": 1000,
            "index_name": "contents",
            "distance": "ip", # "cosine"
            "dimension": 768,
            "exact": False,
            "hnsw_m": 32,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
            "update_docs_per_sec": 0,
            "quantization": "",
        },
        "1m-768-m48-efc200-ef100-ip": {
            "content_path": Path("dataset/passages-c400-jawiki-20230403"),
            "embedding_path": Path("dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage"),
            "num_of_docs": 5555583,
            "index_size": 1000000,
            "bulk_size": 1000,
            "index_name": "contents",
            "distance": "ip", # "cosine"
            "dimension": 768,
            "exact": False,
            "hnsw_m": 48,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
            "update_docs_per_sec": 0,
            "quantization": "",
        },
        "5m-768-m48-efc200-ef100-ip": {
            "content_path": Path("dataset/passages-c400-jawiki-20230403"),
            "embedding_path": Path("dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage"),
            "num_of_docs": 5555583,
            "index_size": 5000000,
            "bulk_size": 1000,
            "index_name": "contents",
            "distance": "ip", # "cosine"
            "dimension": 768,
            "exact": False,
            "hnsw_m": 48,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
            "update_docs_per_sec": 0,
            "quantization": "",
        },
    }
    return DataSetConfig(**setting.get(target_name))


In [ ]:
def run_chroma(config):
    volume_dir = os.getenv("VOLUME_DIR", "./data")
    print(f"Starting {config.chroma_name}... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "run", "-d",
        "--name", config.chroma_name,
        "-p", f"{config.chroma_port}:8000",
        "-e", "IS_PERSISTENT=TRUE",
        "-e", "PERSIST_DIRECTORY=/chroma/chroma",
        # "-v", f"{volume_dir}:/chroma/chroma",
        f"ghcr.io/chroma-core/chroma:{config.chroma_version}"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print("[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def stop_chroma(config):
    print(f"Stopping {config.chroma_name}... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "stop", config.chroma_name
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def prune_docker(config):
    print(f"Cleaning up... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "system", "prune", "-f"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def print_docker_system_df(config):
    docker_cmd = [
        # "sudo",
        "docker", "system", "df"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print(result.stdout)
    else:
        print(result.stderr)


In [ ]:
def print_docker_container_stats(config):
    docker_cmd = [
        # "sudo",
        "docker", "container", "stats", "--no-stream"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    containers = {}
    if result.returncode == 0:
        print(result.stdout)
        for line in result.stdout.split("\n"):
            if line.startswith("CONTAINER") or len(line) == 0:
                continue
            values = line.split()
            containers[values[1]] = {
                "container_id": values[0],
                "cpu": values[2],
                "mem": values[6],
                "mem_usage": values[3],
                "mem_limit": values[5],
                "net_in": values[7],
                "net_out": values[9],
                "block_in": values[10],
                "block_out": values[12],
                "pids": values[13],
            }
    else:
        print(result.stderr)
    return containers


In [ ]:
def create_index(config):
    print(F"Creating {config.index_name}... ", end="")
    client = chromadb.HttpClient(host=config.chroma_host, port=config.chroma_port)
    try:
        collection = client.create_collection(
            name=config.index_name,
            metadata={
                "hnsw:space": config.distance,
                "hnsw:construction_ef": config.hnsw_ef_construction,
                "hnsw:search_ef": config.hnsw_ef,
                "hnsw:M": config.hnsw_m,
            }
        )
        print(f"[OK] -> {collection}")
    except Exception as e:
        print(f"[FAIL] {e}")


In [ ]:
def delete_index(config):
    print(F"Deleting {config.index_name}... ", end="")
    client = chromadb.HttpClient(host=config.chroma_host, port=config.chroma_port)
    try:
        client.delete_collection(name=config.index_name)
        print("[OK]")
    except Exception as e:
        print(f"[FAIL] {e}")


In [ ]:
def print_indices(config):
    client = chromadb.HttpClient(host=config.chroma_host, port=config.chroma_port)
    try:
        collection = client.get_collection(name=config.index_name)
        return {
            "num_of_docs": collection.count(),
        }
    except Exception as e:
        print(f"[FAIL] {e}")
    return {}


In [ ]:
def wait_for_chroma(config, retry_count=60):
    print(f"Waiting for {config.chroma_name}", end="")
    for i in range(retry_count):
        try:
            response = requests.get(f"http://{config.chroma_host}:{config.chroma_port}/api/v1/heartbeat")
            if response.status_code == 200:
                print("[OK]")        
                return
        except:
            pass
        print(".", end="")
        time.sleep(1)
    print("[FAIL]")


In [ ]:
def get_embedding(config, embedding_index, embedding_data, id):
    emb_index = int(id / 100000) * 100000
    if embedding_data is None or embedding_index != emb_index:
        with np.load(config.embedding_path / f"{emb_index}.npz") as data:
            embedding_data = data["embs"]
    embedding = embedding_data[id - emb_index]
    if config.distance == "ip":
        embedding = embedding.astype(np.float32)
        embedding = embedding / np.linalg.norm(embedding)
    return emb_index, embedding_data, embedding


def get_section_values(config, df, min_count=10000):
    tmp_df = df[["id","section"]].groupby("section").count().reset_index()
    tmp_df = tmp_df[tmp_df["id"] >= min_count]
    return tmp_df["section"].values.tolist()


def insert_data(config, max_size, bulk_size, controller=None, query_data=None):
    start_time = time.time()
    client = chromadb.HttpClient(host=config.chroma_host, port=config.chroma_port)
    collection = client.get_collection(name=config.index_name)

    ids = []
    embeddings = []
    metadatas = []
    def send_data(pos):
        print(F"Sending {int(len(ids))} docs ({pos}/{max_size})... ", end="")
        now = time.time()
        try:
            collection.add(
                embeddings=embeddings,
                metadatas=metadatas,
                ids=ids,
            )
            t = time.time() - now
            print(f"[OK] {t}")
            return t
        except Exception as e:
            print(f"[FAIL] {e}")
            return 0

    total_time = 0
    count = 0
    embedding_index = -1
    embedding_data = None
    for content_file in sorted(config.content_path.glob("*.parquet")):
        if count >= max_size:
            break
        df = pd.read_parquet(content_file)
        if query_data is not None:
            query_data["section_values"].extend(get_section_values(config, df))
        for i,row in df.iterrows():
            if count >= max_size:
                break
            embedding_index, embedding_data, embedding = get_embedding(config, embedding_index, embedding_data, row.id)
            count += 1
            ids.append(str(count))
            embeddings.append(embedding.tolist())
            metadatas.append({
                "page_id": row.pageid,
                "rev_id": row.revid,
                # "title": row.title,
                "section": row.section,
                # "text": row.text,
            })
            if len(ids) >= bulk_size:
                total_time += send_data(count)
                ids = []
                embeddings = []
                metadatas = []
                if controller is not None and not controller.run():
                    return

    if len(ids) > 0:
        total_time += send_data(count)

    execution_time = time.time() - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Execution Time: {int(hours):02d}:{int(minutes):02d}:{seconds:02.2f} ({timedelta(seconds=total_time)})")
    return {
        "execution_time": execution_time,
        "process_time": total_time,
    }


In [ ]:
class IndexingController:

    def __init__(self, stop_event):
        self._previous_time = time.time()
        self._stop_event = stop_event

    def run(self):
        if self._stop_event.is_set():
            return False

        wait_time = 1 - ( time.time() - self._previous_time)
        if wait_time > 0:
            # print(f"Waiting for {wait_time} sec")
            time.sleep(wait_time)
        self._previous_time =  time.time()
        return True


def update_data(target_config, stop_event):
    dataset_config = get_dataset_config(target_config)
    if dataset_config.update_docs_per_sec > 0:
        print(f"Starting update for {target_config}")
        insert_data(dataset_config, 
                    max_size=dataset_config.index_size,
                    bulk_size=dataset_config.update_docs_per_sec,
                    controller=IndexingController(stop_event))
        print(f"Stopping update for {target_config}")
    else:
        print(f"No background updates")
        

def start_update(target_config):
    stop_event = multiprocessing.Event()

    p = multiprocessing.Process(target=update_data, args=(target_config, stop_event))
    p.start()

    def stop_update():
        stop_event.set()
        p.join()

    return stop_update


In [ ]:
def search(config, collection, query):
    now = time.time()
    try:
        query_result = collection.query(**query)
        took = time.time() - now
        # print(query_result)

        product_ids = [int(x) for x in query_result.get("ids")[0]]
        scores = [1 - x for x in query_result.get("distances")[0]]
        return took * 1000, len(product_ids), product_ids, scores
    except Exception as e:
        print(f"[FAIL] {e}")
        return -1, -1, [], []


In [ ]:
def search_with_knn_queries(config, output_path, pre_filter=None, max_size=10000, page_size=100, offset=0, max_error_count=100):
    print("Sending knn queries...")
    client = chromadb.HttpClient(host=config.chroma_host, port=config.chroma_port)
    collection = client.get_collection(name=config.index_name)
    start_time = time.time()
    pos = offset
    doc_id = 0
    count = 0
    running = True
    error_count = 0
    with gzip.open(output_path, "wt", encoding="utf-8") as f:
        while running:
            with np.load(config.embedding_path / f"{pos}.npz") as data:
                embedding_data = data["embs"]
            for embedding in embedding_data:
                doc_id += 1
                if count >= max_size:
                    running = False
                    break
                if config.distance == "ip":
                    embedding = embedding.astype(np.float32)
                    embedding = embedding / np.linalg.norm(embedding)
                num_candidates = config.hnsw_ef if config.hnsw_ef > page_size else page_size
                query = {
                    "query_embeddings": embedding.tolist(),
                    "n_results": page_size,
                    "include": [
                        "distances",
                        "metadatas",
                    ],
                }
                if pre_filter is not None:
                    query["where"] = next(pre_filter)
                took, hits, ids, scores = search(config, collection, query)
                # print(f"{took}, {hits}, {ids}, {scores}")
                if took == -1:
                    error_count += 1
                    if error_count >= max_error_count:
                        running = False
                        break
                    continue
                result = {
                    "id": doc_id,
                    "took": took,
                    "hits": hits,
                    "ids": ids,
                    "scores": scores,
                }
                f.write(json.dumps(result, ensure_ascii=False))
                f.write("\n")
                count += 1
                if count % 10000 == 0:
                    print(f"Sent {count}/{max_size} queries.")

            pos += 100000
            if pos > config.num_of_docs:
                pos = 0

    execution_time = time.time() - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Execution Time: {int(hours):02d}:{int(minutes):02d}:{seconds:02.2f}")


In [ ]:
def get_output_filename(version, name, exact):
    filename = f"output/chroma{version.replace('.', '_')}_{name}"
    if exact:
        filename += "_exact"
    filename += ".jsonl.gz"
    return filename


In [ ]:
def print_took_and_total_hits(k, filename, truth_filename):
    query_ids = []
    tooks = []
    hits = []
    df = pd.read_json(filename, lines=True)
    result = {
        "num_of_queries": len(df),
        "took": {
            "mean": df.took.mean(),
            "std": df.took.std(),
            "min": df.took.min(),
            "25%": df.took.quantile(0.25),
            "50%": df.took.quantile(0.5),
            "75%": df.took.quantile(0.75),
            "90%": df.took.quantile(0.9),
            "99%": df.took.quantile(0.99),
            "max": df.took.max(),
        },
        "hits": {
            "mean": df.hits.mean(),
            "std": df.hits.std(),
            "min": df.hits.min(),
            "25%": df.hits.quantile(0.25),
            "50%": df.hits.quantile(0.5),
            "75%": df.hits.quantile(0.75),
            "max": df.hits.max(),
        },
    }
    if os.path.exists(truth_filename):
        df = pd.merge(df,
                      pd.read_json(truth_filename, lines=True)[["id", "ids"]].rename(columns={"ids":"truth_ids"}),
                      on="id", how="inner")
        def get_precision(row):
            size = len(row["truth_ids"])
            if size > k:
                size = k
            return len(set(row["ids"]).intersection(set(row["truth_ids"]))) / size
        df["precision"] = df.apply(get_precision, axis=1)
        result["precision"] = {
            "mean": df.precision.mean(),
            "std": df.precision.std(),
            "min": df.precision.min(),
            "25%": df.precision.quantile(0.25),
            "50%": df.precision.quantile(0.5),
            "75%": df.precision.quantile(0.75),
            "90%": df.precision.quantile(0.9),
            "99%": df.precision.quantile(0.99),
            "max": df.precision.max(),
        }
    print(df.describe().to_markdown())
    return result


In [ ]:
def save_results(target_config, config, results):
    with open("results.json", "wt", encoding="utf-8") as f:
        json.dump({
            "target": target_config,
            "version": config.chroma_version,
            "settings": asdict(config),
            "results": results,
            "timestamp": datetime.now().isoformat(),
        }, f, ensure_ascii=False, default=lambda x: int(x) if isinstance(x, np.int64) else None)


In [ ]:
query_data = {"section_values": []}
results = {}
target_config = os.getenv("TARGET_CONFIG", "100k-768-m32-efc200-ef100-ip")
dataset_config = get_dataset_config(target_config)
pprint.pprint(dataset_config)

In [ ]:
prune_docker(dataset_config)
print(f"<<<Chroma {dataset_config.chroma_version}>>>")
run_chroma(dataset_config)
wait_for_chroma(dataset_config)

In [ ]:
print_docker_container_stats(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
create_index(dataset_config)

In [ ]:
print_docker_container_stats(dataset_config)
print_indices(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
results["indexing"] = insert_data(dataset_config,
                                  max_size=dataset_config.index_size,
                                  bulk_size=dataset_config.bulk_size,
                                  query_data=query_data)


In [ ]:
print_docker_system_df(dataset_config)
print_docker_container_stats(dataset_config)
print_indices(dataset_config)

In [ ]:
results["indexing"]["container"] = print_docker_container_stats(dataset_config)
print_indices(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
for page_size in [10, 100]:
    print(f"page size: {page_size}")
    filename = get_output_filename(dataset_config.chroma_version, f"knn_{page_size}", exact=dataset_config.exact)
    stop_update = start_update(target_config)
    search_with_knn_queries(dataset_config, filename, page_size=page_size, max_size=1000) # warmup
    search_with_knn_queries(dataset_config, filename, page_size=page_size, offset=dataset_config.index_size)
    stop_update()
    results[f"top_{page_size}"] = print_took_and_total_hits(page_size, filename,  f"dataset/ground_truth/{re.sub(r'-m.*', '', target_config)}/knn_{page_size}.jsonl.gz")

In [ ]:
# TODO Filetered Query is very slow...
# def pre_filter_generator():
#     section_values = query_data["section_values"]
#     if len(section_values) > 0:
#         while True:
#             for s in section_values:
#                 yield {
#                     "section": {
#                         "$eq": s
#                     }
#                 }

# results["num_of_filtered_words"] = len(query_data["section_values"])
# for page_size in [10, 100]:
#     print(f"page size: {page_size}")
#     filename = get_output_filename(dataset_config.chroma_version, f"knn_{page_size}_filtered", exact=dataset_config.exact)
#     stop_update = start_update(target_config)
#     search_with_knn_queries(dataset_config, filename, page_size=page_size, max_size=1000, pre_filter=pre_filter_generator()) # warmup
#     search_with_knn_queries(dataset_config, filename, page_size=page_size, offset=dataset_config.index_size, pre_filter=pre_filter_generator())
#     stop_update()
#     results[f"top_{page_size}_filtered"] = print_took_and_total_hits(page_size, filename, f"dataset/ground_truth/{re.sub(r'-m.*', '', target_config)}/knn_{page_size}_filtered.jsonl.gz")

In [ ]:
print_docker_container_stats(dataset_config)
print_indices(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
save_results(target_config, dataset_config, results)

In [ ]:
delete_index(dataset_config)
stop_chroma(dataset_config)